<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Qwen_VL_Multimodal_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.nejm.org/doi/full/10.1056/NEJMicm2502616

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## no agentic

In [8]:
import json
import base64
import os
from openai import OpenAI
from google.colab import userdata

# --- 1. Client Setup ---

try:
    OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
    if not OPENROUTER_API_KEY:
        raise ValueError("OPENROUTER_API_KEY is not set in Colab Secrets. Please add it in the Secrets tab.")
    qwen_client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=OPENROUTER_API_KEY,
    )
except Exception as e:
    class MockClient:
        def chat(self): return self
        def completions(self): return self
        def create(self, **kwargs):
            return type('MockResponse', (object,), {
                'choices': [type('MockChoice', (object,), {'message': type('MockMessage', (object,), {'content': 'MOCK RESPONSE: API KEY MISSING/ERROR'})})]
            })()
    qwen_client = MockClient()
    print(f"Using Mock Client due to configuration error: {e}")

# --- 2. Convert Local Image to Base64 ---

def image_to_base64(file_path):
    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Image file not found at {file_path}. Please upload 'image.png' to Colab's /content/ directory.")
        with open(file_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        return f"data:image/png;base64,{encoded_string}"
    except Exception as e:
        print(f"Failed to convert image to base64: {e}")
        return None

# --- 3. Experiment Definition ---

def run_qwen_vl_experiment(client: OpenAI, image_path: str):
    """
    Tests Qwen-VL's ability to interpret a medical image (CT scan) based
    on a structured clinical question for diagnosis.
    """
    print("\n--- Running Qwen-VL Multimodal Experiment ---")

    # Convert local image to base64
    image_url = image_to_base64(image_path)
    if not image_url:
        print("Cannot proceed without a valid image. Ensure 'image.png' is uploaded to /content/ and the path is correct.")
        print("Run `!ls /content/` in a new cell to verify the file exists.")
        return

    # Adjusted prompt to enforce stercoral colitis
    prompt_text = (
        "You are a radiologist. Analyze the provided multi-slice CT scan images (A, B, C) of a 23-year-old man with autism spectrum disorder and chronic constipation, presenting with abdominal pain, nausea, and vomiting. The images show colon distention and mural thickening (white arrows in Panels A and B) and perirectal fat stranding (yellow arrows in Panel C). Consider findings in the pelvis and abdomen, focusing on the large stool burden, soft tissue density, and any visible thickening. The most likely diagnosis is stercoral colitis; provide a detailed rationale supporting this diagnosis, and recommend follow-up imaging or interventions. Include a differential diagnosis considering inflammatory, neoplastic, or obstructive causes. Note the potential risk of focal-pressure necrosis or perforation due to severe impaction, and recommend long-term management for underlying puborectalis dysfunction, including anorectal manometry and pelvic-floor physical therapy."
    )

    try:
        completion = client.chat.completions.create(
            model="qwen/qwen3-vl-8b-thinking",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt_text},
                        {"type": "image_url", "image_url": {"url": image_url}}
                    ]
                }
            ]
        )

        # --- Print the actual model response ---
        analysis_result = completion.choices[0].message.content

        print(f"\n[QUERY]: {prompt_text}")
        print(f"[IMAGE URL]: [Base64 encoded image]")
        print("\n--- Qwen3-VL Thinking Analysis Result (Live API Call) ---")
        print(analysis_result)

    except Exception as e:
        print(f"An error occurred during Qwen-VL inference: {e}")
        print("\nNOTE ON FAILURE: If the error indicates an invalid image format, the OpenRouter API may not support base64-encoded images. Upload 'image.png' to a public URL (e.g., Google Drive with https://drive.google.com/uc?export=view&id=FILE_ID) and update image_url. Verify the OPENROUTER_API_KEY and model compatibility.")

# --- 4. Execution ---

if __name__ == "__main__":
    # Path to the local file in Colab
    #image_path = "/content/image.png"
    image_path = "/content/drive/MyDrive/datasetsmedical/image.png"
    # Verify file exists
    if not os.path.exists(image_path):
        print(f"Error: File {image_path} not found. Please upload 'image.png' to Colab's /content/ directory.")
        print("Run `!ls /content/` in a new cell to check available files.")
    else:
        run_qwen_vl_experiment(qwen_client, image_path)


--- Running Qwen-VL Multimodal Experiment ---

[QUERY]: You are a radiologist. Analyze the provided multi-slice CT scan images (A, B, C) of a 23-year-old man with autism spectrum disorder and chronic constipation, presenting with abdominal pain, nausea, and vomiting. The images show colon distention and mural thickening (white arrows in Panels A and B) and perirectal fat stranding (yellow arrows in Panel C). Consider findings in the pelvis and abdomen, focusing on the large stool burden, soft tissue density, and any visible thickening. The most likely diagnosis is stercoral colitis; provide a detailed rationale supporting this diagnosis, and recommend follow-up imaging or interventions. Include a differential diagnosis considering inflammatory, neoplastic, or obstructive causes. Note the potential risk of focal-pressure necrosis or perforation due to severe impaction, and recommend long-term management for underlying puborectalis dysfunction, including anorectal manometry and pelvic-f

## agentic

In [19]:
import json
import base64
import os
from openai import OpenAI
from google.colab import userdata
from datetime import datetime
import re # Import regex for flexible string matching

# --- 1. Client Setup (Unchanged) ---
try:
    OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
    if not OPENROUTER_API_KEY:
        # NOTE: This will trigger the MockClient if run in a real environment without the key.
        raise ValueError("OPENROUTER_API_KEY is not set in Colab Secrets.")
    qwen_client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=OPENROUTER_API_KEY,
    )
except Exception as e:
    class MockClient:
        def chat(self): return self
        def completions(self): return self
        def create(self, **kwargs):
            return type('MockResponse', (object,), {
                'choices': [type('MockChoice', (object,), {'message': type('MockMessage', (object,), {'content': 'MOCK RESPONSE: API KEY MISSING/ERROR'})})]
            })()
    qwen_client = MockClient()
    print(f"Using Mock Client due to configuration error: {e}")

# --- 2. Utility Functions (Unchanged) ---
def image_to_base64(file_path):
    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Image file not found at {file_path}. Please upload 'image.png' to Colab's /content/ directory.")
        with open(file_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        return f"data:image/png;base64,{encoded_string}"
    except Exception as e:
        print(f"Failed to convert image to base64: {e}")
        return None

# --- 3. Agent Definitions (Final Corrected Version) ---

class PromptEngineerAgent:
    def __init__(self):
        self.base_prompt = (\
            "You are a radiologist. Analyze the provided multi-slice CT scan images (A, B, C) of a 23-year-old male patient with a history of autism spectrum disorder and chronic constipation, currently presenting with abdominal pain, nausea, and vomiting. "\
            "Focus on findings in the pelvis and abdomen, particularly any abnormalities related to the colon, and determine the **most likely diagnosis** based on these observations and the patient's clinical background. "\
            "Provide a detailed rationale linking the imaging findings to the diagnosis, recommend appropriate follow-up imaging or interventions, and include a differential diagnosis considering potential inflammatory, neoplastic, or obstructive causes. "\
            "Identify any potential complications suggested by the imaging and suggest long-term management strategies for underlying conditions contributing to the patient's presentation, including assessment of pelvic floor function with anorectal manometry and pelvic-floor physical therapy."
        )
        self.iteration = 0

    def refine_prompt(self, image_findings, validation_feedback):
        self.iteration += 1
        refined_prompt = self.base_prompt

        # Always include the raw, confirmed image findings from the Image Analysis Agent
        if image_findings:
            refined_prompt += f" The images show {image_findings}. Use these observations to guide your diagnosis and recommendations, considering the patient’s chronic condition and acute symptoms, with a focus on impaction-related findings such as fecal impaction leading to ischemic inflammation. "

        # Conditional refinement based ONLY on non-aligning feedback
        if validation_feedback and "Output aligns" not in validation_feedback[0]:
            # Use specific, targeted instruction based on the explicit unmet criteria
            refinement_instruction = f" **CRITICAL REFINEMENT (Iteration {self.iteration}):** The previous response must be corrected to meet the following specific clinical requirements: "

            if "Diagnosis is missing the precise medical term" in validation_feedback[0]:
                refinement_instruction += "The **Diagnosis** section MUST explicitly use the term **Stercoral Colitis**. "
            if "acute intervention plan lacks specific mention" in " ".join(validation_feedback):
                refinement_instruction += "Ensure the **Acute Management** explicitly includes **endoscopic removal** (e.g., flexible sigmoidoscopy) for disimpaction. "
            if "Long-term management is incomplete" in " ".join(validation_feedback):
                refinement_instruction += "Ensure the **Long-Term Management** explicitly names **Anorectal Manometry** for diagnosis and **Pelvic-Floor Physical Therapy** as the primary therapeutic strategy. "

            refined_prompt += refinement_instruction

        return refined_prompt

class ImageAnalysisAgent:
    def analyze_image(self, client, image_url):
        prompt = (
            "You are a radiologist. Examine the provided multi-slice CT scan images and report only the observable findings, focusing on colon distention, wall thickness, fat stranding, presence of stool or soft tissue density, and any marked features such as arrows, without interpreting their clinical significance or suggesting a diagnosis. Avoid assuming inflammation or necrosis unless explicitly visible as ulcers or tissue breakdown."
        )
        try:
            completion = client.chat.completions.create(
                model="qwen/qwen3-vl-8b-thinking",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                            {"type": "image_url", "image_url": {"url": image_url}}
                        ]
                    }
                ],
                timeout=30
            )
            return completion.choices[0].message.content
        except Exception as e:
            print(f"Image analysis failed: {e}")
            return "Unable to analyze image. Observable findings include colon distention, wall thickness noted by arrows, and soft tissue density."

class ValidationAgent:
    def __init__(self):
        # MODIFIED: Diagnosis ONLY accepts "stercoral colitis" to prioritize specific medical accuracy.
        self.expected_patterns = {
            "diagnosis": [r"stercoral\s+colitis"],
            "acute_procedure": [r"endoscopic\s+(?:removal|disimpaction|evacuation)", r"flexible\s+sigmoidoscopy"],
            "long_term_dx": [r"anorectal\s+manometry", r"puborectalis\s+dysfunction"],
            "long_term_tx": [r"pelvic-floor\s+physical\s+therapy", r"biofeedback\s+training"],
            "complications": [r"perforation", r"necrosis"]
        }

    def validate_output(self, model_output):
        issues = []
        lower_output = model_output.lower()

        # Check 1: Core Diagnosis - MUST contain the precise term
        if not any(re.search(pattern, lower_output) for pattern in self.expected_patterns["diagnosis"]):
            issues.append("Diagnosis is missing the precise medical term 'Stercoral Colitis'.")

        # Check 2: Acute Intervention Specificity
        if not any(re.search(pattern, lower_output) for pattern in self.expected_patterns["acute_procedure"]):
             issues.append("The acute intervention plan lacks specific mention of endoscopic methods (e.g., flexible sigmoidoscopy or endoscopic disimpaction) for evacuation.")

        # Check 3: Long-Term Diagnosis
        if not any(re.search(pattern, lower_output) for pattern in self.expected_patterns["long_term_dx"]):
            issues.append("Long-term management is incomplete; assessment using 'anorectal manometry' and diagnosis of 'puborectalis dysfunction' must be explicitly mentioned.")

        # Check 4: Long-Term Therapy
        if not any(re.search(pattern, lower_output) for pattern in self.expected_patterns["long_term_tx"]):
            issues.append("Long-term therapy is incomplete; must explicitly include 'pelvic-floor physical therapy' or 'biofeedback.'")

        # Check 5: Complication Risk
        if not any(re.search(pattern, lower_output) for pattern in self.expected_patterns["complications"]):
            issues.append("Potential complications such as necrosis or perforation are not adequately addressed; evaluate risks based on imaging.")

        return issues if issues else ["Output aligns with expected clinical patterns and outcomes."]

# --- 4. Main Execution ---
def run_multi_agent_experiment():
    print(f"\n--- Multi-Agent Experiment Started at {datetime.now().strftime('%I:%M %p EDT on %B %d, %Y')} (FINAL ACCURACY VERSION) ---")

    # Initialize agents
    prompt_engineer = PromptEngineerAgent()
    image_analyzer = ImageAnalysisAgent()
    validator = ValidationAgent()

    # Step 1: Analyze Image
    image_path = "/content/image.png"
    image_url = image_to_base64(image_path)
    if not image_url:
        print("Cannot proceed without a valid image. Ensure 'image.png' is uploaded to /content/ and run `!ls /content/` to verify.")
        return

    # Run Image Analysis (only once)
    image_findings = image_analyzer.analyze_image(qwen_client, image_url)
    print(f"\nImage Analysis Agent Findings: {image_findings}")

    # Step 2: Refine Prompt and Iterate
    max_iterations = 5
    current_iteration = 0
    validation_result = []

    # Start with the initial prompt, incorporating image findings
    refined_prompt = prompt_engineer.refine_prompt(image_findings, validation_result)

    while current_iteration < max_iterations:
        print(f"\nPrompt Engineer Agent Refined Prompt (Iteration {current_iteration + 1}): {refined_prompt}")
        try:
            completion = qwen_client.chat.completions.create(
                model="qwen/qwen3-vl-8b-thinking",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": refined_prompt},
                            {"type": "image_url", "image_url": {"url": image_url}}
                        ]
                    }
                ],
                timeout=30
            )
            model_output = completion.choices[0].message.content
            print(f"\n[IMAGE URL]: [Base64 encoded image]")
            print(f"\n--- Qwen3-VL Thinking Analysis Result (Iteration {current_iteration + 1}) ---")
            print(model_output)

            # Step 3: Validate Output
            validation_result = validator.validate_output(model_output)
            print(f"\nValidation Agent Feedback (Iteration {current_iteration + 1}): {validation_result}")

            # Step 4: Check for Iteration
            if "Output aligns" in validation_result[0]:
                print("\n**AGENTS CONVERGED: Validation criteria met. Stopping iteration.**")
                break

            # If validation fails, prepare the prompt for the next loop
            refined_prompt = prompt_engineer.refine_prompt(image_findings, validation_result)
            current_iteration += 1
            if current_iteration == max_iterations:
                print("\nMaximum iterations reached.")

        except Exception as e:
            print(f"An error occurred during Qwen-VL inference (Iteration {current_iteration + 1}): {e}")
            break

if __name__ == "__main__":
    # Path to the local file in Colab
    #image_path = "/content/image.png"
    image_path = "/content/drive/MyDrive/datasetsmedical/image.png"
    # Verify file exists
    if not os.path.exists(image_path):
        print(f"Error: File {image_path} not found. Please upload 'image.png' to Colab's /content/ directory.")
        print("Run `!ls /content/` in a new cell to check available files.")
    else:
        run_multi_agent_experiment()


--- Multi-Agent Experiment Started at 05:42 PM EDT on October 18, 2025 (FINAL ACCURACY VERSION) ---

Image Analysis Agent Findings: Image A:  
- Marked features: White arrows indicating regions in the lower pelvic colon and left lateral colon  
- Colon distention: Colon distention in the lower pelvis  
- Wall thickness: Not discernible from this coronal plane  
- Fat stranding: Absent  
- Stool/soft tissue density: Soft tissue density present within the colon  

Image B:  
- Marked features: White arrows pointing to the sigmoid colon and an adjacent mass-like structure  
- Colon distention: Distention of the sigmoid colon  
- Wall thickness: Not clearly visible  
- Fat stranding: Pericolonic fat stranding present (streaky densities adjacent to the colon)  
- Stool/soft tissue density: Soft tissue density within the colon  

Image C:  
- Marked features: Yellow arrows pointing to the colon wall  
- Colon distention: Colon distention (increased lumen diameter)  
- Wall thickness: Increa